In [ ]:
# initial setup for any notebook

%load_ext autoreload
%autoreload 2
import sys
import json
sys.path.append("/home/yarcoh/projects/thesis-code4") # go to parent dir

In [ ]:
import numpy as np

from src.optics.channel_simulation2 import ChannelSimulator2
from src.general_methods.visualizer import Visualizer
from src.optics.config_manager import ChannelConfig


# Channel Simulation
here starts the actual channel simulator code.

In [ ]:
conf = ChannelConfig(
    W=0.05,                 # Total bandwidth, estimated [THz]
    Nspans=12,              # The number of spans
    La=80,         # Transmission span [km]
    M_QAM=16,               # QAM order (2,4,16,64,256)
    Ts=1,                   # Symbol period for rrc [unitless]

    # Modulation and Coding:
    Nos=16,                 # Oversampling factor (must be even)
    eta=2,                  # spectral efficiency penalty factor (1,4]
    mu=0.15,                # Dimensionless power scaling factor (RRC)
    bet=0.2,                # roll-off factor
    with_ssf=True,          # whether to use SSF or not
    with_noise=True,        # whether to add noise or not

    # Fiber and Dispersion:
    beta2=-21,              # ps^2/km
    gamma=1.27,             # Nonlinear coefficient in [1/km*W]
    dz=0.2,                 # Z-step, [km] - initial step estimate
    K_T=1.13,                # [unitless]
    chi=0.0461,             # fiber loss coefficient [1/km]

    # general stuff:
    verbose=True,           # whether to print stuff or not
)


In [ ]:
cs: ChannelSimulator2 = ChannelSimulator2(conf)


In [ ]:
Visualizer.print_config(cs.channel_config)

In [ ]:
Visualizer.print_config(cs.cb_configs)

In [ ]:
# ssf config
Visualizer.print_config(cs.blocks[5].config)
print(f'D = {cs.blocks[5].ssf.D:.2e}')

In [ ]:
cs.quick_simulate()

In [ ]:
# 1) modulation
M_QAM = conf.M_QAM
sps = int(np.log2(M_QAM))
print(cs.blocks[0].get_output_names())
message_s_bin, c_in = cs.blocks[0].get_outputs()
bin_txt = np.reshape(message_s_bin, (-1, sps))
print(bin_txt)
Visualizer.plot_constellation_map_with_points(c_in, M_QAM, 'clean before channel')



In [ ]:
# 2) over sampling
cs.blocks[1].get_output_names()
c_in1 = cs.blocks[1].get_outputs()[0]
Visualizer.my_plot(range(50), np.real(c_in1[0:50]), name='zero padded - $ real\{c_1 \} $', function='stem')

In [ ]:
# 3) spectral shaping
cs.blocks[2].get_output_names()
u_in, psi_xi, psi_t = cs.blocks[2].get_outputs()
xi = cs.extra_inputs['xi']
Ns = cs.extra_inputs['Ns']
Nos = conf.Nos
print(f'length of u_in: {len(u_in)}')
print(f'length of xi: {len(xi)}')
zm = range(Ns//2, Ns//2 + Nos*3)
Visualizer.twin_zoom_plot(r'$|\psi(\xi)|^2$', np.abs(psi_xi) ** 2, zm, xi, xlabel=r'$\xi$')
Visualizer.plot_amp_and_phase(xi,u_in,r'$\xi$',r'u(\xi)')

In [ ]:
# 4.5) pre equalize (zero padded)
cs.blocks[3].get_output_names()
u1, b_in1, b_in, b_in_padded = cs.blocks[3].get_outputs()
print(f'length of b_padded: {len(b_in_padded)}')
xi_padded = cs.extra_inputs['xi_padded']
Visualizer.plot_amp_and_phase(xi_padded, b_in_padded, r'$\xi$', r'b(\xi)')

In [ ]:
print(f'Pn = {cs.channel_config.Pn}')

In [ ]:
# 5) inft
cs.blocks[4].get_output_names()
q_in, qb, q_p = cs.blocks[4].get_outputs()
t_padded = cs.extra_inputs['t_padded']
t = cs.extra_inputs['t']
print(f' length of t: {len(t)}')

# Visualizer.twin_zoom_plot('|q(t)|', np.abs(q_in), range(8000, 8200), t_padded, 't')
# Visualizer.twin_zoom_plot('|q_b(t)|', np.abs(qb), range(2000, 2200), t, 't [ps]')
# Visualizer.my_plot(t, np.abs(q_p)**2, name=r'$|q_p(t)|^2$ [W]', xlabel='t [ps]')
dbm_power = 30+10 * np.log10(np.abs(q_p)**2)
Visualizer.my_plot(t, dbm_power, name=r'$|q_p(t)|^2$', xlabel='t [ps]', ylabel='[dBm]')


In [ ]:
# 6) ssf
cs.blocks[5].get_output_names()
qz, q_s, q_pad = cs.blocks[5].get_outputs()

Visualizer.compare_amp_and_phase_log(t, qz, q_p, r'$t$', r'q_z(t)','before and after SSF')
# Visualizer.plot_amp_and_phase(t_padded, q_pad, r'$t_{padded}$', r'q_p(\xi)')

In [ ]:
cs.blocks[6].get_output_names()
b_out_padded, b_out, is_error = cs.blocks[6].get_outputs()

Visualizer.compare_amp_and_phase(xi_padded, b_out_padded, b_in_padded, r'$\xi$', r'b_p(\xi)',"")
Visualizer.plot_amp_and_phase(xi, b_out, r'$\xi$', r'b(\xi)')

In [ ]:
cs.blocks[7].get_output_names()
b_out1, u1_out, u_out = cs.blocks[7].get_outputs()

Visualizer.compare_amp_and_phase(xi,u_out, u_in, r'$\xi$', r'u(\xi)', "")

In [ ]:
cs.blocks[8].get_output_names()
c_out1, c_out = cs.blocks[8].get_outputs()


Visualizer.compare_amp_and_phase(xi, c_out1,c_in1, r'$\xi$', r'\hat c(\xi)', "")
Visualizer.twin_zoom_plot('sampled bits (real)',np.real(c_out),range(0,50),function='stem')
Visualizer.plot_constellation_map_with_points(c_out, M_QAM, 'after depulse shaping')


In [ ]:
cs.blocks[9].get_output_names()
s_out = cs.blocks[9].get_outputs()[0]
Visualizer.print_bits(s_out, sps, 'message after channel')


In [ ]:
cs.block11.verbose = True
cs.block11.calc_ber(s_out,message_s_bin)